# ecog2vec

## Preprocess ECoG data 

Load in the .nwb files, save to .wav files with sr=16000 for compatibility with wav2vec.

In [31]:
# from nwbwidgets import nwb2widget
from pynwb import NWBHDF5IO

path = '/NWB/EFC400/EFC400_B72.nwb' # CHANGE FOR EACH SUBJECT

# Open the NWB file for reading
# with NWBHDF5IO(path, 'r') as io:
#     nwb_file = io.read()

io = NWBHDF5IO(path, load_namespaces=True, mode='r')
nwbfile = io.read()

nwbfile_electrodes = nwbfile.acquisition['ElectricalSeries'].data[:]
print(nwbfile.acquisition['ElectricalSeries'].data[:].shape)

(869888, 256)


In [32]:
import numpy as np
import soundfile as sf
import os

chunk_size = 300000

num_full_chunks = len(nwbfile_electrodes) // chunk_size
last_chunk_size = len(nwbfile_electrodes) % chunk_size

full_chunks = np.split(nwbfile_electrodes[:num_full_chunks * chunk_size], num_full_chunks)
last_chunk = nwbfile_electrodes[num_full_chunks * chunk_size:]

chunks = full_chunks + [last_chunk]

for chunk in chunks:
    print(chunk.shape)

# Loop through the chunks and save them as WAV files
for i, chunk in enumerate(chunks):
    file_name = f'/home/bayuan/Documents/fall23/ecog2vec/ecog/EFC400/EFC400_B72_{i}.wav' # CHANGE FOR EACH SUBJECT
    sf.write(file_name, chunk, 16000)  # adjust as needed

(300000, 256)
(300000, 256)
(269888, 256)


In [2]:
nwbfile

root pynwb.file.NWBFile at 0x139880672181744
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>,
    anin4 <class 'pynwb.base.TimeSeries'>
  }
  devices: {
    R256GridElectrode <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    R256GridElectrode electrodes <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2019, 10, 24, 21, 2, 56, 725378, tzinfo=tzoffset(None, -25200))]
  identifier: EC61_B30
  institution: University of California, San Francisco
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  lab: Chang Lab
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>,
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_description: NWB File
  session_start_time: 2014-06-05 03:37:41-07:00
  stimulus: {
    speaker1 <class 'pynwb.base.TimeSeries'>,
    speaker2 <class 'pynwb.base.TimeSeries'>
  }
  subject: subject abc.ECoGSubject at 0x139880672180064
Fields:
  age__reference: birth
  subject_id: EC61

  timestamps_reference_time: 2014-06-05 03:37:41-07:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>

## Train wav2vec on ecog data

Create a training manifest from the dataset. Train the model with 256-channel inputs

In [33]:
!python3 /home/bayuan/Documents/fall23/fairseq/examples/wav2vec/wav2vec_manifest.py \
    /home/bayuan/Documents/fall23/ecog2vec/ecog/EFC400 \
  --dest /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --ext wav \
  --valid-percent 0.05

In [34]:
!python3 -c 'import argparse; print(argparse.__file__)'
!python3 /home/bayuan/Documents/fall23/fairseq/train.py \
    /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --save-dir /home/bayuan/Documents/fall23/ecog2vec/model \
  --num-workers 6 --fp16 --max-update 400000 --save-interval 1 --no-epoch-checkpoints \
  --arch wav2vec --task audio_pretraining --min-lr 1e-06 --stop-min-lr 1e-09 --optimizer adam --lr 0.00002 --lr-scheduler cosine \
  --conv-feature-layers "[(512, 10, 5), (512, 8, 4), (512, 4, 2), (512, 4, 2), (512, 4, 2), (512, 1, 1), (512, 1, 1)]" \
  --conv-aggregator-layers "[(512, 2, 1), (512, 3, 1), (512, 4, 1), (512, 5, 1), (512, 6, 1), (512, 7, 1), (512, 8, 1), (512, 9, 1), (512, 10, 1), (512, 11, 1), (512, 12, 1), (512, 13, 1)]" \
  --skip-connections-agg --residual-scale 0.5 --log-compression --warmup-updates 500 --warmup-init-lr 1e-07 --criterion wav2vec --num-negatives 10 \
  --max-sample-size 1500000 --skip-invalid-size-inputs-valid-test --max-epoch 25  --batch-size 1 --max-tokens 150000000 --tensorboard-logdir /home/bayuan/Documents/fall23/ecog2vec/runs

/home/bayuan/anaconda3/envs/wav2vec/lib/python3.9/argparse.py
2023-10-18 20:48:07 | INFO | fairseq_cli.train | Args: 
2023-10-18 20:48:08 | INFO | fairseq.models.wav2vec.wav2vec | Wav2VecModel(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(256, 512, kernel_size=(10,), stride=(5,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(512, 512, kernel_size=(8,), stride=(4,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (2-4): 3 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (5-6): 2 x Sequential(
        (0

## Extract embeddings

Load the model, and extract the $c$ vectors from each .nwb file. 

In [39]:
import torch
import fairseq
# from scipy.io import wavfile
import soundfile as sf

cp_path = '/home/bayuan/Documents/fall23/ecog2vec/model/checkpoint_best.pt'#'/path/to/wav2vec.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

wav_path = '/home/bayuan/Documents/fall23/ecog2vec/ecog/EFC400/EFC400_B4_1.wav'

wav_input_16khz, sr = sf.read(wav_path)
wav_input_16khz = wav_input_16khz.T
wav_input_16khz = wav_input_16khz.reshape(1, 256, -1)

wav_input_16khz = torch.from_numpy(wav_input_16khz).to(torch.float)
# print(wav_input_16khz)

# print(sr, wav_input_16khz.shape)
z = model.feature_extractor(wav_input_16khz)
c = model.feature_aggregator(z)

2023-10-18 20:54:16 | INFO | fairseq.models.wav2vec.wav2vec | Wav2VecModel(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(256, 512, kernel_size=(10,), stride=(5,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(512, 512, kernel_size=(8,), stride=(4,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (2-4): 3 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (5-6): 2 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2):

In [41]:
print(c[0][:])
print(c.shape)

tensor([[0.8734, 0.8734, 0.8734,  ..., 0.8673, 0.8721, 0.8783],
        [1.1981, 1.1981, 1.1981,  ..., 1.1979, 1.1879, 1.1902],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.6073, 0.6073, 0.6073,  ..., 0.6121, 0.6120, 0.6096],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1701, 0.1701, 0.1701,  ..., 0.1761, 0.1710, 0.1702]],
       grad_fn=<SliceBackward0>)
torch.Size([1, 512, 1873])
